In [ ]:
import gradio as gr
import torch
from torchvision import models, transforms
from PIL import Image
import requests
import openai
import gradio as gr

openai.api_key = #YOUR OPENAI API KEY
openai.api_base = #Your openai api base
# Define your model class (if needed) or use a predefined model
class ResNet34(torch.nn.Module):
    def __init__(self, num_classes=8):
        super(ResNet34, self).__init__()
        self.network = models.resnet34(weights=None)  # No pre-trained weights
        num_ftrs = self.network.fc.in_features
        self.network.fc = torch.nn.Linear(num_ftrs, num_classes)

    def forward(self, x):
        return self.network(x)

# Load your model
model = ResNet34()
model.load_state_dict(torch.load('model_path', map_location=torch.device('cpu')))
model.eval()

# Define the image preprocessing steps
img_size = 224
imagenet_stats = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
preprocess = transforms.Compose([
    transforms.Resize(img_size),
    transforms.CenterCrop(img_size),
    transforms.ToTensor(),
    transforms.Normalize(*imagenet_stats)
])

# Define the labels and detailed information for your classification
class_info = {
    "Actinic keratosis": {
        "description": "Actinic keratosis is a precancerous skin condition caused by exposure to ultraviolet (UV) rays.",
        "symptoms": "Symptoms include rough, scaly patches on the skin, often on sun-exposed areas.",
        "treatment": "Treatment may involve cryotherapy, topical medications, or photodynamic therapy."
    },
    "Basal cell carcinoma": {
        "description": "Basal cell carcinoma is the most common type of skin cancer, often appearing as a waxy bump.",
        "symptoms": "Symptoms include a shiny, pearly bump or a bleeding or scabbing sore that heals and returns.",
        "treatment": "Treatment may involve surgical excision, Mohs surgery, or topical treatments."
    },
    "Benign keratosis": {
        "description": "Benign keratosis, also known as seborrheic keratosis, is a non-cancerous skin growth.",
        "symptoms": "Symptoms include raised, waxy, or wart-like growths on the skin.",
        "treatment": "Treatment is usually not necessary, but may involve removal if desired for cosmetic reasons."
    },
    "Dermatofibroma": {
        "description": "Dermatofibroma is a benign skin lesion that usually appears as a hard bump.",
        "symptoms": "Symptoms include a firm, round bump on the skin that may be red, pink, or brown.",
        "treatment": "Treatment is usually not necessary, but may involve removal if desired or if causing discomfort."
    },
    "Melanocytic nevus": {
        "description": "Melanocytic nevus, or mole, is a common benign skin growth made up of melanocytes.",
        "symptoms": "Symptoms include small, dark spots on the skin that may be raised or flat.",
        "treatment": "Treatment is usually not necessary, but moles may be removed if changing in size, shape, or color."
    },
    "Melanoma": {
        "description": "Melanoma is a type of skin cancer that develops from melanocytes, often appearing as a dark spot.",
        "symptoms": "Symptoms include changes in the size, shape, color, or feel of an existing mole, or the appearance of a new spot on the skin.",
        "treatment": "Treatment may involve surgical excision, immunotherapy, chemotherapy, or radiation therapy."
    },
    "Squamous cell carcinoma": {
        "description": "Squamous cell carcinoma is a common form of skin cancer, often appearing as a scaly patch or sore.",
        "symptoms": "Symptoms include a persistent, scaly red patch with irregular borders that may bleed or crust.",
        "treatment": "Treatment may involve surgical excision, Mohs surgery, or topical treatments."
    },
    "Vascular lesion": {
        "description": "Vascular lesions are abnormalities of the blood vessels, which can manifest as birthmarks or other skin discolorations.",
        "symptoms": "Symptoms vary depending on the type of vascular lesion and may include red or purple discolorations on the skin.",
        "treatment": "Treatment depends on the type and severity of the lesion, and may include laser therapy or surgical removal."
    }
}

def predict(image):
    # Preprocess the image
    image = Image.fromarray(image)
    image = preprocess(image).unsqueeze(0)  # Add batch dimension

    # Predict
    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)
        prediction = list(class_info.keys())[predicted.item()]
        info = class_info[prediction]

    return prediction, info['description'], info['symptoms'], info['treatment']


home_page_content = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Skin Health</title>

    <!-- Swiper CSS -->
    <link rel="stylesheet" href="https://unpkg.com/swiper/swiper-bundle.min.css">

    <!-- Custom styles -->
    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            color: #fff; /* Set text color to white */
            background-color: #000;
        }


        .swiper-container {
        width: 100%;
        height: auto; /* Change height to auto */
        overflow: hidden; /* Hide overflow */
        }

        .swiper-slide {
        text-align: center;
        font-size: 18px;
        display: flex;
        justify-content: center;
        align-items: center;
        }

        .swiper-slide img {
        max-width: 100%; /* Set maximum width to 100% */
        height: auto; /* Auto-adjust height */
        }

        .description {
            max-width: 600px;
            margin: 20px auto;
            text-align: center;
            padding: 20px;
            background-color: #000;
            border-radius: 10px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
        }

        h2 {
            margin-bottom: 10px;
        }

        p {
            margin-bottom: 20px;
        }

        .section {
            margin-top: 20px;
            display: flex;
            justify-content: space-around;
            align-items: center;
            flex-wrap: nowrap;
            overflow-x: auto;
            scrollbar-width: none; /* Firefox */
            -ms-overflow-style: none; /* IE and Edge */
            scrollbar-color: transparent; /* Firefox */
        }

        .section::-webkit-scrollbar {
            display: none; /* Safari and Chrome */
        }

        .section-item {
        width: calc(45% - 20px); /* Adjust width to fit 5 items per row */
        margin: 5px;
        position: relative;
        overflow: hidden;
        flex-shrink: 0;
        }

        .section-item img {
            transition: transform 0.3s ease-in-out;
        }

        .section-item:hover img {
            transform: scale(1.1);
        }

        .overlay {
            position: absolute;
            top: 0;
            left: 0;
            width: 100%;
            height: 100%;
            background: rgba(0, 0, 0, 0.5);
            display: flex;
            justify-content: center;
            align-items: center;
            opacity: 0;
            transition: opacity 0.3s ease-in-out;
        }

        .section-item:hover .overlay {
            opacity: 1;
        }

        .overlay-content {
            text-align: center;
            color: #fff;
        }
        .sec {
            display: flex;
            justify-content: space-between;
             align-items: flex-start;
        }

        .sec-item {
              width: 100%;
              display: flex;
        }

        .content {
            width: 50%;
            padding: 20px;
        }

        .content img {
             max-width: 100%;
        }
    </style>
</head>
<body>

<!-- Skin Health Section -->
<div class="description">
    <h2>Skin Health</h2>
    <p>Healthy skin is essential for overall well-being. Explore the importance of skin health and how to maintain it.</p>
</div>

<div style="max-width: 100%; height: auto; text-align: center; ">
    <img src="https://www.reequil.com/cdn/shop/articles/Skin_care_tips_for_different_types_of_complexions_reequil.jpg?v=1652772499&width=1100" alt="Skin Health" style="max-width: 100%; height: auto; display: block; margin: 0 auto;">
</div>

<!-- Autoimmune Diseases Section -->
<div class="description">
    <h2>Autoimmune Diseases</h2>
    <p>Learn about common autoimmune skin diseases and their symptoms.</p>
</div>

<div class="section">
    <div class="section-item">
        <img src="https://www.isdin.com/en-US/blog/wp-content/uploads/2023/04/2023_Keratosis_Header-1-1170x600.png" alt="Autoimmune Disease 1">
        <div class="overlay">
            <div class="overlay-content">
                <h3>Actinic keratosis</h3>
                <p>A rough, scaly patch on the skin caused by years of sun exposure.
Actinic keratoses usually affects older adults. Reducing sun exposure can help reduce risk.
It is most common on the face, lips, ears, back of hands, forearms, scalp and neck. The rough, scaly skin patch enlarges slowly and usually causes no other signs or symptoms. A lesion may take years to develop.
Because it can become cancerous, it's usually removed as a precaution.</p>
            </div>
        </div>
    </div>
    <div class="section-item">
        <img src="https://miiskin.com/wp-content/uploads/2020/04/basal-cell-carcinoma-types.png" alt="Autoimmune Disease 2">
        <div class="overlay">
            <div class="overlay-content">
                <h3>Basal cell carcinoma</h3>
                <p>Basal cell carcinoma (BCC), previously known as basal cell epithelioma, is the most common cancer in Humans. BCC mostly arises on sun-damaged skin and rarely develops on the mucous membranes or palms and soles. Basal cell carcinoma is usually a slow-growing tumor for which metastases are rare.</p>
            </div>
        </div>
    </div>
    <div class="section-item">
        <img src="https://i0.wp.com/post.medicalnewstoday.com/wp-content/uploads/sites/3/2022/05/slide1-Seborrheic-keratosis.jpg?w=1155" alt="Autoimmune Disease 3">
        <div class="overlay">
            <div class="overlay-content">
                <h3>Bengin Keratosis</h3>
                <p>A non-cancerous skin condition that appears as a waxy brown, black or tan growth.
A seborrhoeic keratosis is one of the most common non-cancerous skin growths in older adults. While it's possible for one to appear on its own, multiple growths are more common.
Seborrheic keratosis often appears on the face, chest, shoulders or back. It has a waxy, scaly, slightly elevated appearance.</p>
            </div>
        </div>
    </div>
    <div class="section-item">
        <img src="https://lavanamed.com/wp-content/uploads/2022/02/Vascular-lesions.jpg" alt="Autoimmune Disease 4">
        <div class="overlay">
            <div class="overlay-content">
                <h3>Vascular lesion</h3>
                <p>Definition. Vascular lesions are abnormal growths or malformations in the blood vessels, which can occur in various parts of the body. They can be congenital or acquired and may result from injury, infection, or other underlying medical conditions.</p>
            </div>
        </div>
    </div>
    <div class="section-item">
        <img src="https://www.usdermatologypartners.com/wp-content/uploads/2023/10/AdobeStock_261932205_Resized-for-Hero.jpg" alt="Autoimmune Disease 5">
        <div class="overlay">
            <div class="overlay-content">
                <h3>Melanoma</h3>
                <p>Melanoma is a kind of skin cancer that starts in the melanocytes. Melanocytes are cells that make the pigment that gives skin its color. The pigment is called melanin. Melanoma typically starts on skin that's often exposed to the sun</p>
            </div>
        </div>
    </div>
</div>

<!-- Recovery Stories Section -->
<div class="description">
    <h2>Recovery Stories</h2>
    <p>Discover inspiring recovery stories from individuals overcoming skin conditions.</p>
</div>

<div class="sec">
    <div class="sec-item">
        <div class="content">
            <img src="https://media.post.rvohealth.io/wp-content/uploads/2020/09/psoriasis_doesnt_define_you-732x549-thumbnail-732x549.jpg" alt="Patient Image">
        </div>
        <div class="content">
            <h3>My Psoriasis Is Not Me</h3>
            <p>Around 2001, after seeing that last dermatologist, I stopped with everything, summoning a Buddha-like indifference to my disease. I told myself that the only way to control the symptoms was to let go of the need to control them. It was the only treatment I hadn't tried -- detachment. I set my disease on a shelf like a book I had already read and reread.</p>
            <p>Of course, having a toddler around at the time meant I couldn't think about tending to my skin. Having a husband who doesn't notice the surface of things -- he walks around oblivious to the crumbs in his mustache and mustard stains on his shirt -- means not having to wince if his hand brushes my knee.</p>
            <p>Happily, my symptoms have receded somewhat, probably an effect of the sense of well-being that comes from a good night's sleep, regular exercise, and my children's laughter. My gynecologist suggested that age-related hormonal changes might also have driven the psoriasis underground.</p>
        </div>
    </div>
</div>


<!-- Swiper JS -->
<script src="https://unpkg.com/swiper/swiper-bundle.min.js"></script>

<!-- Initialize Swiper -->
<script>
    var swiper = new Swiper('.swiper-container', {
        // Optional parameters
        loop: true,
        slidesPerView: 1,
        spaceBetween:0,
        centeredSlides: true,

        // If we need pagination
        pagination: {
            el: '.swiper-pagination',
            clickable: true,
        },
    });
</script>

</body>
</html>

"""

def chatbot_response(message, history):
    # Predefined responses based on the input query
    formatted_history = []
    for user_message, assistant_message in history:
        formatted_history.append({"role": "user", "content": user_message})
        formatted_history.append({"role": "assistant", "content": assistant_message})

    formatted_history.append({"role": "user", "content": message})

    completion = openai.ChatCompletion.create(
        model="meta-llama/Meta-Llama-3-70B-Instruct",
        messages=formatted_history,
        temperature=0.7
    )
    print(formatted_history)
    ai_response = completion["choices"][0]["message"]["content"]
    return ai_response

# Define the combined interface
with gr.Blocks(title="Skin Disease Detection with Chatbot") as combined_interface:
    with gr.Tab("Home"):
      gr.HTML(home_page_content)
    with gr.Tab("Skin Disease Detection"):
        inputs = gr.Image(label="Upload an image of a skin lesion")
        outputs = [
            gr.Textbox(label="Prediction"),
            gr.Textbox(label="Description"),
            gr.Textbox(label="Symptoms"),
            gr.Textbox(label="Treatment")
        ]
        gr.Interface(fn=predict, inputs=inputs, outputs=outputs, title="Skin Disease Detection",
                     description="Upload an image of a skin lesion to get a prediction and detailed information about the disease.")

    with gr.Tab("Chatbot"):
        gr.ChatInterface(chatbot_response)

# Launch the combined interface
combined_interface.launch(share=True, inbrowser=True, auth=('user','123'), auth_message=("check your <i>email</i> for <strong>Login details</strong>"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f2070dd708f06c461a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
